In [62]:
import pandas as pd
import ast # Crucial for safely converting the string representation of a list/dict to a Python object
import numpy as np


  

if __name__ == '__main__':

    try:
        df_lmsys = pd.read_csv('lmsys_chat_500.csv')
        

        if 'language' in df_lmsys.columns:
             df_lmsys = df_lmsys[df_lmsys['language'] == 'English'].copy()
             
        if 'model' in df_lmsys.columns:
             df_lmsys.drop(columns=['model'], inplace=True)
             

    except FileNotFoundError:
        print("Error: 'lmsys_chat_500.csv' not found. Ensure the file is accessible.")
        exit()


In [63]:
df_lmsys

,Unnamed: 0,conversation_id,conversation,turn,language,openai_moderation,redacted
0,0,33f01939a744455c869cb234afca47f1,[{'content': 'how can identity protection serv...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
1,1,1e230e55efea4edab95db9cb87f6a9cb,"[{'content': ""Beside OFAC's selective sanction...",6,English,"[{'categories': {'harassment': False, 'harassm...",False
2,2,0f623736051f4a48a506fd5933563cfd,"[{'content': ""You are the text completion mode...",1,English,"[{'categories': {'harassment': False, 'harassm...",False
3,3,e5c923a7fa3f4893beb432b4a06ef222,[{'content': 'The sum of the perimeters of thr...,2,English,"[{'categories': {'harassment': False, 'harassm...",False
4,4,8ad66650dced4b728de1d14bb04657c1,"[{'content': ""What is the type of the variable...",1,English,"[{'categories': {'harassment': False, 'harassm...",False
...,...,...,...,...,...,...,...
494,494,455f28ef8aa0418394d38414e906b1c2,"[{'content': ""You are the text completion mode...",1,English,"[{'categories': {'harassment': False, 'harassm...",False
495,495,67899abdb342447286beb54f007ca3c5,[{'content': 'Give me an introduction over 200...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
497,497,146331884935416a894c807b43faa32f,[{'content': 'Give me a list of digital produc...,1,English,"[{'categories': {'harassment': False, 'harassm...",True
498,498,1175f7c3c0f0495597f4ba639b7db7d9,"[{'content': ""You are the text completion mode...",1,English,"[{'categories': {'harassment': False, 'harassm...",False


In [64]:
import pandas as pd

# -------------------------------
# Reuse the extract_brace_blocks function
# -------------------------------
def extract_brace_blocks(text):
    """
    Extracts top-level {...} blocks from a string
    and returns them as a list of strings.
    """
    results = []
    stack = []
    start = None
    for i, ch in enumerate(text):
        if ch == '{':
            if not stack:
                start = i
            stack.append(ch)
        elif ch == '}':
            if stack:
                stack.pop()
                if not stack and start is not None:
                    results.append(text[start:i+1])
                    start = None
    return results

# -------------------------------
# Apply to the entire DataFrame
# -------------------------------
df_lmsys["extracted_blocks"] = df_lmsys["conversation"].apply(extract_brace_blocks)

# -------------------------------
# Optional: preview
# -------------------------------
print(df_lmsys[["conversation", "extracted_blocks"]].head())

# -------------------------------
# Optional: save to JSON file
# -------------------------------
import json
all_blocks = df_lmsys["extracted_blocks"].to_list()
with open("all_extracted_blocks.json", "w") as f:
    json.dump(all_blocks, f, indent=2)


                                        conversation  \
0  [{'content': 'how can identity protection serv...   
1  [{'content': "Beside OFAC's selective sanction...   
2  [{'content': "You are the text completion mode...   
3  [{'content': 'The sum of the perimeters of thr...   
4  [{'content': "What is the type of the variable...   

                                    extracted_blocks  
0  [{'content': 'how can identity protection serv...  
1  [{'content': "Beside OFAC's selective sanction...  
2  [{'content': "You are the text completion mode...  
3  [{'content': 'The sum of the perimeters of thr...  
4  [{'content': "What is the type of the variable...  


In [65]:
df_lmsys['conversation']=df_lmsys['extracted_blocks']
del df_lmsys['extracted_blocks']

In [70]:
type(df_lmsys['openai_moderation'][0])

str

In [75]:
import pandas as pd
import ast
import json

# Extract top-level { ... } blocks
def extract_brace_blocks(text):
    results = []
    stack = []
    start = None
    for i, ch in enumerate(text):
        if ch == '{':
            if not stack:
                start = i
            stack.append(ch)
        elif ch == '}':
            if stack:
                stack.pop()
                if not stack and start is not None:
                    results.append(text[start:i+1])
                    start = None
    return results

# Parse each block safely
def parse_moderation_column(text):
    blocks = extract_brace_blocks(text)
    parsed = []
    for block in blocks:
        try:
            parsed.append(ast.literal_eval(block))
        except Exception:
            # Keep raw string if still malformed
            parsed.append(block)
    return parsed

# Apply to your DataFrame
df_lmsys["moderation_list"] = df_lmsys["openai_moderation"].apply(parse_moderation_column)

# Optional: save as JSON
all_data = df_lmsys["moderation_list"].to_list()
with open("openai_moderation_cleaned.json", "w") as f:
    json.dump(all_data, f, indent=2)

# Preview first row
print(json.dumps(df_lmsys["moderation_list"].iloc[0], indent=2))


[
  {
    "categories": {
      "harassment": false,
      "harassment/threatening": false,
      "hate": false,
      "hate/threatening": false,
      "self-harm": false,
      "self-harm/instructions": false,
      "self-harm/intent": false,
      "sexual": false,
      "sexual/minors": false,
      "violence": false,
      "violence/graphic": false
    },
    "category_scores": {
      "harassment": 9.212334e-07,
      "harassment/threatening": 3.6271535e-07,
      "hate": 8.254962e-07,
      "hate/threatening": 1.331843e-08,
      "self-harm": 1.9257865e-07,
      "self-harm/instructions": 6.5028324e-08,
      "self-harm/intent": 2.7525962e-07,
      "sexual": 2.564017e-06,
      "sexual/minors": 4.523274e-07,
      "violence": 2.2942773e-05,
      "violence/graphic": 1.9713439e-06
    },
    "flagged": false
  },
  {
    "categories": {
      "harassment": false,
      "harassment/threatening": false,
      "hate": false,
      "hate/threatening": false,
      "self-harm": false,


In [84]:
df_lmsys['openai_moderation']=df_lmsys['moderation_list']
del df_lmsys['moderation_list']

In [98]:
len(df_lmsys['conversation'][0])

2

In [88]:
import re
import json
import pandas as pd

# -------------------------------
# Step 1: Extract top-level { ... } blocks
# -------------------------------
def extract_brace_blocks(text):
    results = []
    stack = []
    start = None
    for i, ch in enumerate(text):
        if ch == '{':
            if not stack:
                start = i
            stack.append(ch)
        elif ch == '}':
            if stack:
                stack.pop()
                if not stack and start is not None:
                    results.append(text[start:i+1])
                    start = None
    return results

# -------------------------------
# Step 2: Convert string block to dict safely
# -------------------------------
def safe_block_to_dict(s):
    s = s.strip().replace("\n", " ")
    # Replace single quotes around keys/values with double quotes for JSON
    s = re.sub(r"(?<=\{|,)\s*'([^']+)'\s*:", r'"\1":', s)
    s = re.sub(r":\s*'([^']+)'(?=,|\})", r': "\1"', s)
    try:
        return json.loads(s)
    except:
        return None  # skip malformed blocks

# -------------------------------
# Step 3: Apply to a list of strings
# -------------------------------
def clean_list_of_malformed_strings(lst):
    cleaned = []
    for s in lst:
        blocks = extract_brace_blocks(s)
        for b in blocks:
            d = safe_block_to_dict(b)
            if d:
                cleaned.append(d)
    return cleaned

# -------------------------------
# Step 4: Apply to your DataFrame
# -------------------------------
df_lmsys["fixed_conversation_cleaned"] = df_lmsys["conversation"].apply(clean_list_of_malformed_strings)

# Optional: save as JSON
all_data = df_lmsys["fixed_conversation_cleaned"].to_list()
with open("cleaned_conversations_final.json", "w") as f:
    json.dump(all_data, f, indent=2)

# Preview
import pprint
pprint.pprint(df_lmsys["fixed_conversation_cleaned"].iloc[0])

[{'content': 'how can identity protection services help protect me against '
             'identity theft',
  'role': 'user'},
 {'content': 'Identity protection services can help protect you against '
             'identity theft in several ways:\n'
             '\n'
             '1. Monitoring: Many identity protection services monitor your '
             'credit reports, public records, and other sources for signs of '
             'identity theft. If they detect any suspicious activity, they '
             'will alert you so you can take action.\n'
             '2. Credit freeze: Some identity protection services can help you '
             'freeze your credit, which makes it more difficult for thieves to '
             'open new accounts in your name.\n'
             '3. Identity theft insurance: Some identity protection services '
             'offer insurance that can help you recover financially if you '
             'become a victim of identity theft.\n'
             '4. Assist

In [95]:
df_lmsys['conversation']=df_lmsys['fixed_conversation_cleaned']
del  df_lmsys['fixed_conversation_cleaned']


In [103]:
df_lmsys["conversation"] = df_lmsys["conversation"].apply(lambda x: x[:2] if isinstance(x, list) else None)
df_lmsys["moderation"] = df_lmsys["openai_moderation"].apply(lambda x: x[:2] if isinstance(x, list) else None)

In [ ]:
df_lmsys


,Unnamed: 0,conversation_id,conversation,turn,language,openai_moderation,redacted,conversation_top2,moderation_top2,moderation
0,0,33f01939a744455c869cb234afca47f1,[{'content': 'how can identity protection serv...,1,English,"[{'categories': {'harassment': False, 'harassm...",False,[{'content': 'how can identity protection serv...,"[{'categories': {'harassment': False, 'harassm...","[{'categories': {'harassment': False, 'harassm..."
1,1,1e230e55efea4edab95db9cb87f6a9cb,[{'content': 'Beside OFAC's selective sanction...,6,English,"[{'categories': {'harassment': False, 'harassm...",False,[{'content': 'Beside OFAC's selective sanction...,"[{'categories': {'harassment': False, 'harassm...","[{'categories': {'harassment': False, 'harassm..."
2,2,0f623736051f4a48a506fd5933563cfd,[{'content': 'You are the text completion mode...,1,English,"[{'categories': {'harassment': False, 'harassm...",False,[{'content': 'You are the text completion mode...,"[{'categories': {'harassment': False, 'harassm...","[{'categories': {'harassment': False, 'harassm..."
3,3,e5c923a7fa3f4893beb432b4a06ef222,[{'content': 'The sum of the perimeters of thr...,2,English,"[{'categories': {'harassment': False, 'harassm...",False,[{'content': 'The sum of the perimeters of thr...,"[{'categories': {'harassment': False, 'harassm...","[{'categories': {'harassment': False, 'harassm..."
4,4,8ad66650dced4b728de1d14bb04657c1,[{'content': 'What is the type of the variable...,1,English,"[{'categories': {'harassment': False, 'harassm...",False,[{'content': 'What is the type of the variable...,"[{'categories': {'harassment': False, 'harassm...","[{'categories': {'harassment': False, 'harassm..."
...,...,...,...,...,...,...,...,...,...,...
494,494,455f28ef8aa0418394d38414e906b1c2,[{'content': 'You are the text completion mode...,1,English,"[{'categories': {'harassment': False, 'harassm...",False,[{'content': 'You are the text completion mode...,"[{'categories': {'harassment': False, 'harassm...","[{'categories': {'harassment': False, 'harassm..."
495,495,67899abdb342447286beb54f007ca3c5,[{'content': 'Give me an introduction over 200...,1,English,"[{'categories': {'harassment': False, 'harassm...",False,[{'content': 'Give me an introduction over 200...,"[{'categories': {'harassment': False, 'harassm...","[{'categories': {'harassment': False, 'harassm..."
497,497,146331884935416a894c807b43faa32f,[{'content': 'Give me a list of digital produc...,1,English,"[{'categories': {'harassment': False, 'harassm...",True,[{'content': 'Give me a list of digital produc...,"[{'categories': {'harassment': False, 'harassm...","[{'categories': {'harassment': False, 'harassm..."
498,498,1175f7c3c0f0495597f4ba639b7db7d9,[{'content': 'You are the text completion mode...,1,English,"[{'categories': {'harassment': False, 'harassm...",False,[{'content': 'You are the text completion mode...,"[{'categories': {'harassment': False, 'harassm...","[{'categories': {'harassment': False, 'harassm..."


In [ ]:
del df_lmsys['conversation_id']
del df_lmsys['moderation_top2']
del df_lmsys['language']
del df_lmsys['model']
del df_lmsys['conversation_top2']


In [114]:
del df_lmsys['turn']
del df_lmsys['moderation']

In [115]:
df_lmsys.to_csv('lmsys_chat_500_cleaned.csv', index=False)

In [116]:
df_lmsys

,Unnamed: 0,conversation,openai_moderation,redacted
0,0,[{'content': 'how can identity protection serv...,"[{'categories': {'harassment': False, 'harassm...",False
1,1,[{'content': 'Beside OFAC's selective sanction...,"[{'categories': {'harassment': False, 'harassm...",False
2,2,[{'content': 'You are the text completion mode...,"[{'categories': {'harassment': False, 'harassm...",False
3,3,[{'content': 'The sum of the perimeters of thr...,"[{'categories': {'harassment': False, 'harassm...",False
4,4,[{'content': 'What is the type of the variable...,"[{'categories': {'harassment': False, 'harassm...",False
...,...,...,...,...
494,494,[{'content': 'You are the text completion mode...,"[{'categories': {'harassment': False, 'harassm...",False
495,495,[{'content': 'Give me an introduction over 200...,"[{'categories': {'harassment': False, 'harassm...",False
497,497,[{'content': 'Give me a list of digital produc...,"[{'categories': {'harassment': False, 'harassm...",True
498,498,[{'content': 'You are the text completion mode...,"[{'categories': {'harassment': False, 'harassm...",False
